In [1]:
from math import asin
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [23]:
df = pd.read_csv('gb_sem_8_hm.csv')

In [24]:
df.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [25]:
df.shape

(10000, 3)

In [26]:
df.USER_ID.nunique()

6324

In [30]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,425.508225
std,2890.590115,4095.565566
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,47119.000000


In [31]:
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    3664
2    1890
3     570
4     159
5      36
6       5
Name: VARIANT_NAME, dtype: int64

In [33]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [34]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)

In [35]:
df_new.describe()

,USER_ID,REVENUE
count,3664.000000,3664.000000
mean,5008.342249,447.914099
std,2897.829839,4170.348384
min,4.000000,0.000000
25%,2478.500000,0.000000
50%,5007.500000,0.000000
75%,7536.250000,0.000000
max,9998.000000,46813.000000


In [36]:
df_new.isna().sum()

USER_ID         0
VARIANT_NAME    0
 REVENUE        0
dtype: int64

In [38]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [37]:
control = df_new[df_new['VARIANT_NAME'] == 'control'].copy(deep=True)
treatment = df_new[df_new['VARIANT_NAME'] == 'variant'].copy(deep=True)

In [40]:
control.describe()

,USER_ID,REVENUE
count,1834.000000,1834.000000
mean,5049.845692,445.771936
std,2879.580387,4080.217859
min,12.000000,0.000000
25%,2591.500000,0.000000
50%,5047.500000,0.000000
75%,7615.000000,0.000000
max,9998.000000,45658.000000


In [41]:
treatment.describe()

,USER_ID,REVENUE
count,1830.000000,1830.000000
mean,4966.748087,450.060945
std,2916.198883,4259.877877
min,4.000000,0.000000
25%,2393.500000,0.000000
50%,4973.000000,0.000000
75%,7418.750000,0.000000
max,9990.000000,46813.000000


рекомендуем отказаться от внедрения проверяемых изменений